In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Get data

In [2]:
# prepare data (can add or remove some columns)
# cleaned_data_dir = relative_path_from_file(__file__, "../../data")
cleaned_data_dir = '../../data/'
games = pd.read_csv(f'{cleaned_data_dir}/games_cleaned.csv')
mechanics = pd.read_csv(f'{cleaned_data_dir}/mechanics_cleaned.csv')
subcategories = pd.read_csv(f'{cleaned_data_dir}/subcategories_cleaned.csv')
themes = pd.read_csv(f'{cleaned_data_dir}/themes_cleaned.csv')
# print(games.dtypes)

games.set_index("BGGId", inplace=True)

games = games.drop(columns=['NumOwned', 'NumWant', 'NumWish', 'NumWeightVotes', 'StdDev', 'BayesAvgRating', 'GoodPlayers', 'ImagePath', 'NumUserRatings', 'NumComments', 'NumAlternates', 'NumExpansions', 'NumImplementations', 'IsReimplementation', 'NumImplementations', 'Family'])
games = games.drop(games.select_dtypes(include=['float']).columns, axis=1)
games = games[games.columns.drop(list(games.filter(regex='Rank')))]
# Drop all columns containing floats

string_dict = {}
for idx, row in games.loc[:, ~games.columns.isin(['BGGId'])].iterrows():
  string_dict[idx] = str(row["Description"]).lower()
  temp = ['_'.join(column_name.split()) + '_' + str(value) for column_name, value in row.items() if value != 0 and column_name!="Description"]
  string = ' ' + ' '.join(temp) + ' '
  string_dict[idx] += string

for table in [mechanics, subcategories, themes]:
  table.set_index("BGGId", inplace=True)
  for idx, row in table.loc[:, ~table.columns.isin(['BGGId'])].iterrows():
      # Create a list of column names where the corresponding value is 1
      columns_with_ones = ['_'.join(column_name.split()) for column_name, value in row.items() if value == 1]
      
      # Combine the column names into a string, with the format "Column1, Column3"
      string = ' '.join(columns_with_ones) + ' '
      
      # Add the string to the list
      string_dict[idx] += str(string)

# remove possible multiple whitespaces
for idx in string_dict.keys():
    string_dict[idx] = ' '.join(string_dict[idx].split())

games = pd.read_csv(f'{cleaned_data_dir}/games_cleaned.csv')
# return games, string_dict

## Compute TFidf

In [3]:
# just to be safe sort it by id
strings = [value for (id, value) in sorted(string_dict.items(), key=lambda x: x[0])]
tfidf_matrix = TfidfVectorizer().fit_transform(strings)
# return 

## Compute Similarity Matrix

In [4]:
similarity_scores = x = cosine_similarity(tfidf_matrix).astype(np.float32)

: 

: 

In [ ]:
print(similarity_scores.shape)

(21919, 21919)


In [ ]:
np.save("../../app/db/similarity_matrix", similarity_scores)